### Introducción a la Investigación Operativa y la Optimización

### • Clase 5  - Métodos Cuasi-Newton

**Nazareno Faillace Mullen - Departamento de Matemática, FCEN, UBA**

La idea, como en el caso del Método de Newton, es aproximar $f$ mediante una expresión cuadrática. Sin embargo, en los métodos cuasi-Newton, se utiliza, en vez de $Hf(x)$, una matriz $B_k$ que sea simétrica definida positiva. La idea es que $B_k$ aproxime a $Hf(x_k)$. 

Como hemos visto, en el método de Newton la dirección de descenso (si $Hf(x_k)\succ 0$) viene dada por:

$$-Hf(x_k)^{-1}\nabla f(x^{k})$$

Dos enfoques: <br>
• Aproximar $Hf(x_k)$ con matrices $B_k$ <br>
• Aproximar $Hf(x_k)^{-1}$ con matrices $H_k$ [$\leftarrow$ trabajamos con este]

### Algoritmo general de un método Cuasi-Newton

Dados: $f,\; x^0 \in \mathbb{R}^n, \; H_0\in \mathbb{R}^{n\times n}\;\;\text{definida positiva},\; \varepsilon > 0, \; k_{MAX} > 0$ <br>
$k = 0$ <br>
REPETIR  mientras $\lVert \nabla f(x_k) \rVert > \varepsilon$ y $k<k_{MAX}$:<br>
&nbsp;&nbsp;&nbsp;&nbsp; Definir $d_k = -H_k\nabla f(x_k)$<br>
&nbsp;&nbsp;&nbsp;&nbsp; Obtener $t_k$ (sección áurea, Armijo o Wolfe)<br>
&nbsp;&nbsp;&nbsp;&nbsp; Hacer $x_{k+1} = x_k + t_kd_k$ <br>
&nbsp;&nbsp;&nbsp;&nbsp; Determinar $H_{k+1}$ definida positiva<br>
&nbsp;&nbsp;&nbsp;&nbsp; $k=k+1$<br>
DEVOLVER $x_k$

__Obs:__ si $H_k = I \rightarrow$ Método del Gradiente <br>
si $H_k = Hf(x_k)^{-1} \rightarrow$ Método de Newton

__Obs:__ como $H_0$ podemos tomar la identidad, ya que es definida positiva. También podría tomarse $Hf(x_0)$ si es definida positiva.

Sean $y_k = \nabla f(x_{k+1}) - \nabla f(x_k)$ y $s_k = x_{k+1} - x_k$, se demuestra que una propiedad que debe satisfacer $H_{k+1}$ definida por el algoritmo es que:
$$H_{k+1}y_j = s_j \quad \forall j=0,1,\dots,k$$

### Broyden (Mala)

$$H_{k+1} = H_k + \dfrac{(s_k-H_ky_k)(s_k-H_ky_k)^T}{y_k^T(s_k-H_ky_k)}$$

### Método DFP (Davindon, Fletcher y Powell)

$$H_{k+1} = H_k + \dfrac{s_k(s_k)^T}{(y_k)^Ts_k} - \dfrac{H_ky_k(y_k)^TH_k}{(y_k)^TH_ky_k}$$

### Método BFGS (Broyden, Fletcher, Goldfarb y Shanno)

$$H_{k+1} = H_k + \left(1 + \dfrac{(y_k)^TH_ky_k}{(s_k)^Ty_k}\right)\dfrac{s_k(s_k)^T}{(s_k)^Ty_k} - \dfrac{s_k(y_k)^TH_k + H_ky_k(s_k)^T}{(s_k)^Ty_k} $$

In [26]:
def broydenMala(hk, sk, yk):
    res = hk + (np.outer((sk-hk@yk),(sk-hk@yk).T)) / (yk.T@(sk-hk@yk))
    return res

In [34]:
def metodoDFP(hk, sk, yk):
    res = hk + (np.outer(sk,sk.T))/(yk.T@sk)
    res = res -  (hk@np.outer(yk,(yk.T))@hk)/(yk.T@ hk@yk)
    return res

In [19]:
def metodoBFGS(hk, sk, yk):
    res = hk + (1+(yk.T@hk@yk)/(sk.T@yk)) * (sk@sk.T)/(sk.T@yk)
    res = res - ((sk @ (yk.T) @ hk) + hk@yk@(sk.T))/(sk.T@yk)
    return res

In [35]:
def cuasiNewtonParaCuadraticas(A, b, x0, h0, eps, maxIter):
    itera = 0
    xViejo = x0
    hk = h0
    dkViejo = A@xViejo+b
    xs = [xViejo]
    while(np.linalg.norm(dkViejo)>eps and itera<maxIter):
        dkNuevo = A@xViejo+b
        dkNuevo = -hk@dkNuevo 
        tk =  (dkNuevo @ dkNuevo.T)/(dkNuevo.T@A@dkNuevo)
        xNuevo = xViejo + dkNuevo * tk
        sk = (xNuevo - xViejo)
        yk = A@sk
        hk = metodoDFP(hk, sk, yk) #cheqear si broyden es lo mejor
                                      
        itera = itera + 1
        xs.append(xNuevo)
        
        xViejo = xNuevo
        dkViejo = dkNuevo
    return(xViejo, itera, xs)

### Importante

Como en general se trabaja con vectores columna:<br>
• $\mathbf{u^T}\mathbf{v} = <\mathbf{u}, \mathbf{v}>\rightarrow$ `u @ v` <br> 

• $ {\displaystyle \mathbf{u} \mathbf{v^T} = \mathbf {u} \otimes \mathbf {v} ={\begin{bmatrix}u_{1}v_{1}&u_{1}v_{2}&\dots &u_{1}v_{n}\\u_{2}v_{1}&u_{2}v_{2}&\dots &u_{2}v_{n}\\\vdots &\vdots &\ddots &\vdots \\u_{m}v_{1}&u_{m}v_{2}&\dots &u_{m}v_{n}\end{bmatrix}}}\rightarrow$ `np.outer(u, v)`

In [22]:
u = np.array([1,-2,3])
v = np.array([-1,1/2,0])
print(u@v.T)
print(np.outer(u, v))

-2.0
[[-1.   0.5  0. ]
 [ 2.  -1.  -0. ]
 [-3.   1.5  0. ]]


## Ejercicios

1. Implementar los tres métodos de Cuasi-Newton para el caso de **funciones cuadráticas** $f(x)=\frac{1}{2}x^T A x + bx + c$ con $A\succ 0$. Debe tomar como input la matriz $A$, el vector $b$, el vector inicial $x_0$ y la cantidad máxima de iteraciones $k_{MAX}$. Utilizar que para este tipo de funciones sabemos que: <br>
• $\nabla f(x_k) = Ax_k +b$ <br>
• $t_k = -\dfrac{(Ax_k +b)^T d_k}{(d_k)^TAd_k}$ <br>
• $y_k=\nabla f(x_{k+1}) - \nabla f(x_k)=(Ax_{k+1} + b) - (Ax_k+b) = A(x_{k+1} - x_k)= As_k$
2. Testear los tres métodos con las funciones cuadráticas dadas por $f(x)=\frac{1}{2}x^T A_i x$ para cada una de las $A_i$ que figuran debajo. Comparar el número de iteraciones de cada uno con el de Gradiente Conjugado.
3. Para $A_4$ graficar el recorrido que realiza cualquier de los métodos y el de Gradiente Conjugado.
4. Implementar los tres métodos para funciones en general y aplicarlos para hallar el minimizador de la función de resta de exponenciales. Comparar con el Método de Newton Modificado y con Gradiente Conjugado. Se pueden elegir, por ejemplo, estos puntos iniciales: <br>
• $x_0=(1,0)$ <br>
• $x_0 =(2, 1.5)$ <br>
• $x_0=(0.5, 0.5)$ <br>
• $x_0=(0,0)$ <br>

In [23]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [24]:
import numpy as np
from numpy.linalg import norm, eigvals, solve
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

A1 = np.array([[8, 3, 3, 6, 5, 4, 4, 3, 6, 3],
             [3, 4, 2, 2, 2, 1, 3, 3, 3, 2],
             [3, 2, 5, 2, 1, 2, 4, 2, 4, 1],
             [6, 2, 2, 6, 3, 2, 4, 2, 4, 2],
             [5, 2, 1, 3, 5, 4, 1, 2, 4, 3],
             [4, 1, 2, 2, 4, 5, 1, 2, 5, 2],
             [4, 3, 4, 4, 1, 1, 6, 2, 4, 2],
             [3, 3, 2, 2, 2, 2, 2, 4, 4, 2],
             [6, 3, 4, 4, 4, 5, 4, 4, 8, 3],
             [3, 2, 1, 2, 3, 2, 2, 2, 3, 4]])

A2 = np.array([[2, 0, 1, 0, 1],
               [0, 2, 1, 1, 1],
               [1, 1, 3, 1, 1],
               [0, 1, 1, 1, 0],
               [1, 1, 1, 0, 2]])

A3 = np.diag(np.random.randint(1, 5, 100))

A4 = np.array([[10, 0], [0, 1]])

In [38]:
cuasiNewtonParaCuadraticas(A2, np.zeros(5), np.ones(5), 1*np.eye(5), 0.000001, 10000)

(array([-3.70406159e-16, -1.17589257e-16, -8.81919427e-17,  3.17490994e-16,
         2.35178514e-16]),
 6,
 [array([1., 1., 1., 1., 1.]),
  array([ 0.23219814,  0.04024768, -0.34365325,  0.42414861,  0.04024768]),
  array([ 0.06976744, -0.1627907 , -0.06578073,  0.30232558,  0.02458472]),
  array([-0.04380242, -0.14196956, -0.04939422,  0.21714818,  0.11711712]),
  array([-0.03278689, -0.1420765 , -0.04918033,  0.19672131,  0.12021858]),
  array([ 8.74300632e-16,  2.77555756e-16,  2.08166817e-16, -7.49400542e-16,
         -5.55111512e-16]),
  array([-3.70406159e-16, -1.17589257e-16, -8.81919427e-17,  3.17490994e-16,
          2.35178514e-16])])

In [15]:
np.eye(10)*5

array([[5., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 5., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 5., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 5., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 5., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 5., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 5., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 5., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 5., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 5.]])

In [16]:
def resta_exponenciales(x):
    s1 = np.exp(-x[0] ** 2 - x[1] ** 2)
    s2 = np.exp(-(x[0] - 1) ** 2 - (x[1] - 1) ** 2)
    return (s1 - s2) * 2

def a_forma_cuadratica(A):
    """
    Transforma la función dada por (1/2)(x^T A x) a una función dada en términos de x1 y x2 para que sea posible 
    graficar sus curvas de nivel en R2.
    A tiene que ser una matriz de 2x2
    """
    def forma_cuadratica(x):
        return 0.5*(A[0,0]*(x[0]**2) + (A[0,1]+A[1,0])*x[0]*x[1] + A[1,1]*(x[1]**2))
    return forma_cuadratica

def plot_fun(f, limites, points=None):
    """
    f : función a graficar
    limites : toma una tupla (x1,x2,y1,y2) de los límites del gráfico: grafica en el dominio [x1,x2] x [y1,y2]
    points : lista de puntos a graficar sobre la superficie; se ingresa como una lista de tuplas (x,y,z) 
    """
    init_notebook_mode(connected=True)

    x = np.linspace(limites[0], limites[1], 1000)
    y = np.linspace(limites[2], limites[3], 1000)
    X, Y = np.meshgrid(x, y)
    Z = f((X, Y))
    data = [go.Surface(x=x, y=y, z=Z)]
    if points is not None:
        for p in points:
            data.append(go.Scatter3d(x=[p[0]], y=[p[1]], z=[p[2]], mode='markers'))
    fig = go.Figure(data=data)
    iplot(fig)

%matplotlib notebook
def graficar_recorrido(f, limites, recorrido=None, levels=10):
    """ 
    Función que grafica curvas de nivel y, opcionalmente, el recorrido de un método.
    f : es la función a graficar (tiene que ir de R2 en R)
    limites : es una lista o tupla de números: [a,b,c,d]. Va a graficar la función en el cuadrado [a,b] x [c,d]
    recorrido : acepta una lista de arrays bidimensionales para graficar el recorrido de un método
    levels : cantidad de curvas de nivel a graficar
    """
    plt.figure()
    x = np.linspace(limites[0], limites[1], 1000)
    y = np.linspace(limites[2], limites[3], 1000)
    X, Y = np.meshgrid(x, y)
    Z = f((X, Y))
    plt.contour(X,Y,Z, cmap='plasma', levels=levels)
    if recorrido is not None:
        x_coords = [x[0] for x in recorrido]
        y_coords = [x[1] for x in recorrido]
        plt.plot(x_coords, y_coords, marker='o', lw=2, ms=8)
    plt.tight_layout()
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

In [17]:
# EJEMPLO PARA GRAFICAR EL RECORRIDO EN EL EJERCICIO 2
x_opt, iteraciones, recorrido = metodo_gradiente(A4, np.zeros(2), np.ones(2), 100)
f = a_forma_cuadratica(A4)
graficar_recorrido(f, [-1, 1, -1, 1], recorrido)

NameError: name 'metodo_gradiente' is not defined